In [19]:
%load_ext autoreload
%autoreload 2

from dataset import TokenTypesDataset
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
train_dataset = TokenTypesDataset(folder="../rnn_tokentype_data/train")
val_dataset = TokenTypesDataset(folder="../rnn_tokentype_data/validation", train=False, vocabs=(train_dataset.token2idx, train_dataset.idx2token), max_length=train_dataset.max_length)
test_dataset = TokenTypesDataset(folder="../rnn_tokentype_data/test", train=False, vocabs=(train_dataset.token2idx, train_dataset.idx2token), max_length=train_dataset.max_length)
assert val_dataset.vocab_size == train_dataset.vocab_size == test_dataset.vocab_size
assert val_dataset.max_length == train_dataset.max_length == test_dataset.max_length

In [21]:
from model import LanguageModel
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = LanguageModel(train_dataset).to(device)

In [22]:
for bs in [1, 4]:
    indices = torch.randint(high=train_dataset.vocab_size, size=(bs, train_dataset.max_length))
    lengths = torch.randint(low=1, high=train_dataset.max_length + 1, size=(bs, ))
    logits = model(indices, lengths)
    assert logits.shape == (bs, lengths.max(), train_dataset.vocab_size)

In [25]:
import torch.nn as nn
from torch.utils.data import DataLoader
from train import train

In [26]:
NUM_EPOCHS = 20

torch.manual_seed(42)

model = LanguageModel(train_dataset).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
scheduler = None
criterion = nn.CrossEntropyLoss(ignore_index=train_dataset.pad_id)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

train(model, optimizer, scheduler, train_loader, test_loader,  NUM_EPOCHS)

torch.save(model.state_dict(), "vanilla_rnn.pth")

Training 1/20:   0%|          | 0/11028 [00:00<?, ?it/s]

KeyboardInterrupt: 